In [10]:
import geopandas as gpd

,avg_m2_price_rent,mean_price_rent,geometry
0,724.346716,45426.664221,"MULTIPOLYGON (((317501.141 6657001.314, 317490..."
1,883.362808,45554.482535,"POLYGON ((347889.496 6647694.723, 347900.553 6..."


In [42]:
path = "/home/gk/vscode/rita_project_4/industrial-location/api/app/provisions_data/saint-petersburg_prov/houses_price_demo.parquet"
gdf = gpd.read_parquet(path)
gdf.rename(columns={"mean_price_rent": "price"}, inplace=True)
gdf['price'] = gdf['price'].round(0).astype(int)
gdf.to_parquet(path)

In [38]:
import pandas as pd
from tqdm import tqdm

# "policlinics",
services = ["hospitals", "kindergartens", "multifunctional_centers", "schools"]
# 'Spb', 
cities = ['Shakhty', 'Perm', 'Tomsk']

def get_prov_spb(city, service=None):
    if city == 'Санкт-Петербург':
        gdf_provision_service = gpd.read_parquet(
                    f"../provisions_data/{city}_prov/{service}.parquet"
                )
    else:
        gdf_provision_service = gpd.read_parquet(
                f"../provisions_data/{city}_prov/provision_buildings.parquet"
            )
    gdf_provision_service.geometry = gdf_provision_service.representative_point()
    
    return gdf_provision_service


def get_house_prices(city):
    h_gdf = gpd.read_parquet(f'../provisions_data/{city}_prov/houses_price_demo.parquet')
    h_gdf.to_crs(4326, inplace=True)
    return h_gdf

def join_prov_prices(h_gdf, gdf_provision_service, cols=["total_provision_assessment", "geometry"]):
    if cols:
        gdf_provision_service = gdf_provision_service[cols] 
    return h_gdf.sjoin(
        gdf_provision_service,
        predicate="intersects",
        how="inner",
    )\
    .reset_index(drop=True)\
    .drop(columns=['index_right'])
            
def save_gdf(h_gdf, city):
    h_gdf.to_parquet(f'../provisions_data/{city}_prov/houses_price_demo_prov.parquet')

def filter_cols(df):
    return df[df.columns[pd.Series(df.columns).str.contains("_provison_value|geometry")]]

for city in tqdm(cities):
    # if city == 'Санкт-Петербург':
    #     for service in services:
    #         gdf_provision_service = get_prov_spb(city, service)
    #         h_gdf = get_house_prices(city).iloc[:, :3]
    #         h_gdf = join_prov_prices(h_gdf, gdf_provision_service)
    #         h_gdf.rename(columns={"total_provision_assessment": f"P_{service}"}, inplace=True)
    #         save_gdf(h_gdf, city)
    # else:
    gdf_provision_service = get_prov_spb(city)
    gdf_provision_service = filter_cols(gdf_provision_service)
    
    for col in gdf_provision_service.columns:
        if "_provison_value" in col:
            gdf_provision_service.rename(columns={col: 'P_' + col.split('_provison_value')[0]}, inplace=True)
    h_gdf = get_house_prices(city)
    h_gdf = join_prov_prices(h_gdf, gdf_provision_service, cols=None)
    save_gdf(h_gdf, city)
        


100%|██████████| 3/3 [00:01<00:00,  1.92it/s]


In [39]:
h_gdf.head(2)

,display_name,m2_price,price,geometry,P_kindergartens,P_schools,P_hospitals,P_policlinics,P_multifunctional_centers
0,"Tomsk, городской округ Томск, Tomsk Oblast, Si...",812,32480,"POLYGON ((84.97932 56.50976, 84.97911 56.50962...",0.000000,0.65,1.0,1.0,0.0
1,"Tomsk, городской округ Томск, Tomsk Oblast, Si...",812,32480,"POLYGON ((84.98024 56.51050, 84.98000 56.51060...",0.769231,0.00,1.0,1.0,0.0
